In [31]:
import numpy as np
from collections import Counter
import porter as p

# Exercice 1

*Données pour test basique*

In [83]:
docs=["the new home has been saled on top forecasts",
     "the home sales rise in july",
     "there is an increase in home sales in july",
     "july encounter a new home sales rise"]

stopWords=["the","a","an","on","behind","under","there","in"]

index = {}
for i in range(len(docs)):
    index[i] = dict(Counter(map(p.stem, [word for word in (str.lower(docs[i])).split() if word not in stopWords])))           
            
indexInverse = {}
for numDoc, dico in index.items():
    for word, tf in dico.items():
        if(word not in indexInverse):
            indexInverse[word]= {}
        indexInverse[word][numDoc] = tf

## 1)

Afin d'optimiser le calcul du score, on utilisera l'index inversé qui permet de retrouver les documents à partir des termes (et donc à partir de la requete) plutôt que d'utiliser l'index qui nous oblige à parcourir tous les documents.

## 2)

On considère ici un modèle booleen simple où une requete du type "mot1 mot2 mot3" équivaut à "mot1^mot2^mot3" i.e un document contenant tous les mots de la requete. <br/>
Dans cette situation, on peut simplement récupérer les documents contenants le i-ème mot dans un l'ensemble i et prendre l'intersection de tous ces ensembles.

In [5]:
#Modele booleen
req_str="home sales top top" #requete avec format "humain"
#découpe la requete, applique une stemmatisation et supprime les doublons (car modèle booleen)
req = list(np.unique(list(map(p.stem, req_str.split())))) 
#représentation du résultat comme unensemble de documents
res=set(index)
for stem in req:
    res=res.intersection(indexInverse[stem])#On récupère l'intersection des documents contenant un mot de la requete
res

{0}

# Exercice 2

# I.

Notation : <br/>
- $w_{t,d}$ est le poids d'un terme $t$ dans un document $d$ et $w_{t,q}$ le poids d'un terme $t$ dans une requete $q$.<br/>
- $tf_{t,d}$ (resp. $tf_{t,q}$) correspond au *term frequency* du terme $t$  dans le document $d$ (resp. la requete $q$).<br/>
- $idf_t$ correspond à l'*inverse document frequency* du terme $t$ dans l'ensemble de la collection/du corpus considéré<br/><br/>

La méthode **getWeightsForDoc** (resp. **getWeightsForStem**) permet de récupérer tous les poids $w_{t,d}$ pour un document $d$ donné (resp. un terme $t$ donné).<br/>
La méthode **getWeightsForQuery** permet de récupérer tous les poids $w_{t,q}$ pour une requete $q$ donnée.

In [33]:
class Weighter():
    """
    Classe abstraite représentant le squelette d'une pondération.
    """
    def __init__(self, index, indexInverse):
        """
        Permet d'initaliser en indiquant l'index et l'index inversé utilisé
        
        :param index: L'index considéré
        :param indexInverse: L'index inversé considéré

        :type index: dict[int : dict[string: int]]
        :type indexInverse: dict[string : dict[int: int]]
        """
        self.index = index
        self.indexInverse = indexInverse
        self.idf = {}
        self.norm = {}
        
    def getIdf(self, stem):
        """
        Permet de récupérer l'idf (index inverse frequency) d'une terme dans le corpus considéré.
        
        :param stem: Le terme dont l'idf doit être calculé

        :type stem: string
        
        :return: L'idf
        :rtype: float 
        """
        #Si l'idf du terme n'a jamais été calculé, on le calul et on l'enregistre avant de renvoyer la valeur
        if(stem not in self.idf):
            if(stem not in indexInverse):
                df = 0
            else:
                df = len(indexInverse[stem])
            self.idf[stem] = np.log((1+len(self.index)) / (1+df))
        return self.idf[stem]
    
    def getWeightsForDoc(self, idDoc):
        """
        Permet de récupérer les poids des termes du document indiqué.
        
        :param idDoc: L'id du document à considérer

        :type idDoc: int
        
        :return: Les poids des différents termes.
        :rtype: dict[string : number]
        """
        pass
    
    def getWeightsForStem(self, stem):
        """
        Permet de récupérer les poids du terme indiqué dans chaque document du corpus.
        
        :param stem: Le terme à considérer

        :type stem: string
        
        :return: Les poids du terme dans chaque document.
        :rtype: dict[int : number]
        """
        pass
    
    def getWeightsForQuery(self, query):
        """
        Permet de récupérer les poids de chaque terme dans la requete indiqué.
        
        :param query: La requete à considérer

        :type query: string
        
        :return: Les poids du terme dans la requete.
        :rtype: dict[string : number]
        """
        pass
    
    def getNormDoc(self, docId):
        """
        Permet de récupérer la norme d'un document vectorisé.
        
        :param docId: L'id du document à considérer

        :type docId: int
        
        :return: La norme du document vectorisé
        :rtype: float
        """
        #Si on n'a jamais calculé la norme du document, on le fait puis on l'enregistre avant de la retourner
        if(docId not in self.norm):
            docWeights = self.getWeightsForDoc(docId)#On récupère le poids de chaque terme du document dans un dictionnaire
            self.norm[docId] = np.linalg.norm(list(docWeights.values()))#On transforme en list/vecteur pour calculer la norme
            #Remarque : la norme d'un vecteur à N dimensions est égale à la norme de ce vecteur auquel on rajoute M dimensions nulles. (||[1,2]|| = ||[0,0,1,2]||)
        return self.norm[docId]
    
    def getNormQuery(self, query):
        """
        Permet de récupérer la norme d'une requete vectorisé.
        
        :param query: Requete à considérer

        :type query: string
        
        :return: La norme du document vectorisé
        :rtype: float
        """
        #On récupère le poids de chaque terme de la requete dans un dictionnaire
        reqWeights = self.getWeightsForQuery(query)
        #On transforme en list/vecteur pour calculer la norme
        #Remarque : la norme d'un vecteur à N dimensions est égale à la norme de ce vecteur auquel on rajoute M dimensions nulles. (||[1,2]|| = ||[0,0,1,2]||)
        return np.linalg.norm(list(reqWeights.values()))
    
    def getStemsFromQuery(query):
        """
        Permet de récupérer l'ensemble des termes d'une requete.
        
        :param query: Requete à considérer

        :type query: string
        
        :return: liste des termes de la requete
        :rtype: list[string]
        """
        return list(np.unique(list(map(p.stem, query.split())))) 

$w_{t,d} = tf_{t,d}$ et $w_{t,q} = 1$ si $t \in q$

In [69]:
class Weighter1(Weighter):
    """
    CF class Weighter
    """
    def getWeightsForDoc(self, idDoc):
        """
        CF class Weighter
        """
        return self.index[idDoc] #correspond aux tfs des termes du document
    
    def getWeightsForStem(self, stem):
        """
        CF class Weighter
        """
        return self.indexInverse[stem] if stem in self.indexInverse else {} #correspond aux tfs du terme pour chaque document
    
    def getWeightsForQuery(self, query):
        """
        CF class Weighter
        """
        #récupère chaque stem de la requete et lui attribut une valeur de 1 car np.unique est utilisé
        return dict(Counter(np.unique(list(map(p.stem, query.split())))))

$w_{t,d} = tf_{t,d}$ et $w_{t,q} = tf_{t,q}$ si $t \in q$

In [68]:
class Weighter2(Weighter):
    """
    CF class Weighter
    """
    def getWeightsForDoc(self, idDoc):
        """
        CF class Weighter
        """
        return self.index[idDoc] #correspond aux tfs des termes du document
    
    def getWeightsForStem(self, stem):
        """
        CF class Weighter
        """
        return self.indexInverse[stem] if stem in self.indexInverse else {} #correspond aux tfs du terme pour chaque document
    
    def getWeightsForQuery(self, query):
        """
        CF class Weighter
        """
        #récupère chaque stem de la requete et lui attribut une valeur égale à son nombre
        return dict(Counter(list(map(p.stem, query.split()))))

$w_{t,d} = tf_{t,d}$ et $w_{t,q} = idf_t$ si $t \in q$

In [67]:
class Weighter3(Weighter):
    """
    CF class Weighter
    """
    def getWeightsForDoc(self, idDoc):
        """
        CF class Weighter
        """
        return self.index[idDoc] #correspond aux tfs des termes du document
    
    def getWeightsForStem(self, stem):
        """
        CF class Weighter
        """
        return self.indexInverse[stem] if stem in self.indexInverse else {} #correspond aux tfs du terme pour chaque document
    
    def getWeightsForQuery(self, query):
        """
        CF class Weighter
        """
        #On récupère chaque terme de manière unique
        req=np.unique(list(map(p.stem, query.split())))
        res={}
        for stem in req:
            res[stem] = self.getIdf(stem)#On récupère l'idf du terme considéré
        return res

$w_{t,d} = 1 + ln(tf_{t,d})$ et $w_{t,q} = idf_t$ si $t \in q$

In [66]:
class Weighter4(Weighter):
    """
    CF class Weighter
    """
    def getWeightsForDoc(self, idDoc):
        """
        CF class Weighter
        """
        #On associe à chaque terme le poids 1 + log(tf)
        res={}
        for stem in index[idDoc]:
            res[stem] = 1+np.log(index[idDoc][stem])
        return res
    
    def getWeightsForStem(self, stem):
        """
        CF class Weighter
        """
        #On associe à chaque document le poids 1 + log(tf)
        res={}
        if stem in indexInverse:
            for doc in indexInverse[stem]:
                res[doc] = 1+np.log(indexInverse[stem][doc])
        return res
    
    def getWeightsForQuery(self, query):
        """
        CF class Weighter
        """
        #On récupère chaque terme de manière unique
        req=np.unique(list(map(p.stem, query.split())))
        res={}
        for stem in req:
            res[stem] = self.getIdf(stem)#On récupère l'idf du terme considéré
        return res

$w_{t,d} = (1 + ln(tf_{t,d})) \times idf_t$ et $w_{t,q} = (1 + ln(tf_{t,q})) \times idf_t$ si $t \in q$

In [65]:
class Weighter5(Weighter):
    """
    CF class Weighter
    """
    def getWeightsForDoc(self, idDoc):
        """
        CF class Weighter
        """
        #On associe à chaque terme le poids 1 + log(tf) * idf
        res = {}
        for stem in index[idDoc]:
            idf = self.getIdf(stem)
            res[stem] = (1+np.log(index[idDoc][stem])) * idf
        return res
    
    def getWeightsForStem(self, stem):
        """
        CF class Weighter
        """
        #On associe à chaque document le poids 1 + log(tf) * idf
        res={}
        if stem in indexInverse:
            idf = self.getIdf(stem)
            for doc in indexInverse[stem]:
                res[doc] = (1+np.log(indexInverse[stem][doc])) * idf
        return res
    
    def getWeightsForQuery(self, query):
        """
        CF class Weighter
        """
        #On associe à chaque terme le poids 1 + log(tf) * idf
        tfs=dict(Counter(list(map(p.stem, query.split())))) #On calcul le tf pour les termes de la requete
        res={}
        for stem in tfs:
            idf = self.getIdf(stem)
            res[stem] = (1+np.log(tfs[stem]))*idf
        return res

# II.

In [39]:
class IRModel():
    """
    Squelette de classe permettant d'utiliser un modèle de Recherche d'Information.
    """
    def __init__(self, weighter):
        """
        Permet d'initialiser en indiquant le systeme de pondération utilisé
        
        :param weighter: Instance d'une classe Weighter

        :type weighter: Weighter
        """
        self.weighter = weighter
    
    def getScores(query):
        """
        Permet de retourner le score de chaque document selon le modèle choisi.
        
        :param query: La requete à considérer

        :type query: string
        
        :return: Le score de chaque document
        :rtype: dict[int: number]
        """
        pass
    
    def getRanking(query):
        """
        Permet de retourner un classement de document par pertinence selon le modèle choisi.
        
        :param query: L'id du document à considérer

        :type query: string
        
        :return: liste des identifiants des documents du corpus triés par ordre décroissant de pertinence
        :rtype: list[int]
        """
        #On transforme le dictionnaire key:valeur en list de couple (key, valeur)
        #On trie ensuite selon la valeur par ordre décroissant
        dictToList = list(zip(query.keys(),query.values())) # Transforme un dict[key:val] en une list[(key,val)]
        sortedList = sorted(dictToList, key=lambda e : e[1], reverse=True)#Permet de trier la liste selon les valeur
        return np.array(sortedList)[:,0]#Permet de récupérer seulement les identifiants des dictionnaires

#### Remarque :

1) Le poids d'un terme n'appartenant pas à la requete sera toujours nul. Ainsi, le produit scalaire entre le vecteur de la requete et un vecteur de document ne prendra pas en compte les termes ne se trouvant pas dans la requete (multiplication par 0).<br/>

Ainsi, on ne retournera pas les documents ayant un score nul (rapidité d'execution). La norme de chaque vecteur sera calculée la première fois que cela est nécessaire et sera gardée en mémoire pour la suite.<br/><br/>

2) Le cosinus ($= \frac{A \cdot B}{||A|| \times ||B||}$) sera toujours négatifcar les normes le sont et le produit scalaire aussi.<br/>
En effet, les vecteurs auront toujours des poids positifs. La somme des produits de poids positifs sera aussi positif.

In [57]:
class Vectoriel(IRModel):
    """
    Permet de représenter le modèle vectoriel en RI.
    """
    def __init__(self, weighter, normalized = True):
        """
        Permet d'initialisé en indiquant le mode de calcul utilisé entre le produit scalaire (normalized = False) et le cosinus (normalized = True)
        
        :param normalized: Booleen indiquant le mode de calcul à effectuer

        :type normalized: boolean
        """
        super().__init__(weighter)
        self.normalized = normalized
        
    def getScores(self, query):
        """
        CF class IRModel
        """
        if(self.normalized == True):#calcul du cosinus
            return self.getScoresNormalized(query)
        else:#calcul du produit scalaire
            return self.getScoresNotNormalized(query)
        
    def getScoresNormalized(self, query):
        """
        Permet de récupérer le score en utilisant le cosinus entre les représentations vectorielles
        
        param query: La requete à considérer

        :type query: string
        
        :return: Le score de chaque document
        :rtype: dict[int: number]
        """
        prodScalaires = self.getScoresNotNormalized(query)#Permet de récupérer les produits scalaires avec les différents documents
        res={}
        for docId,prod in prodScalaires.items():#Pour chaque produit scalaire, on divise par le produit des norm
            res[docId] = prod/(self.weighter.getNormDoc(docId)*self.weighter.getNormQuery(query))
        return res
    
    def getScoresNotNormalized(self, query):
        """
        Permet de récupérer le score en utilisant le produit scalaire entre les représentations vectorielles des documents
        
        param query: La requete à considérer

        :type query: string
        
        :return: Le score de chaque document
        :rtype: dict[int: number]
        """
        reqWeights = self.weighter.getWeightsForQuery(query) #On récupère les poids de la requetes
        res={}
        #Pour chaque terme de la requete (avec son poids associé)
        for stem,weightStem in reqWeights.items():
            docWeights = self.weighter.getWeightsForStem(stem)  #On récupère les poids du terme dans chaque document
            #Pour chaque document contenant le terme courant (et son poids associé)
            for docId,weightDoc in docWeights.items():
                if(docId not in res):#Si le document n'a pas encore été rencontré
                    res[docId] = weightStem*weightDoc#On ajoute le document avec comme valeur le produit des poids
                else:
                    res[docId] += weightStem*weightDoc#On ajoute le produit des poids à l'ancienne valeur
        return res    

# --- TEST ---

In [58]:
w=Weighter1(index, indexInverse)
t=Vectoriel(w, True)
t.getScores("rise encounter")

{3: 0.5773502691896258, 1: 0.26726124191242434}

In [27]:
query={1:0,2:20}
np.array(sorted(list(zip(query.keys(),query.values())), key=lambda e : e[0], reverse=False))[:,0]
sorted(list(zip(query.keys(),query.values())), key=lambda e : e[1], reverse=True)

[(2, 20), (1, 0)]

In [29]:
query={1:30,2:20}
l=list(zip(query.keys(),query.values()))
l

[(1, 30), (2, 20)]

In [38]:
np.array(sorted(list(zip(query.keys(),query.values())), key=lambda e : e[1], reverse=False))[:,0]

array([2, 1])

In [18]:
d={1:10,2:20,3:30}
for (k,v) in d.items():
    print("k:",k," | v:",v)

k: 1  | v: 10
k: 2  | v: 20
k: 3  | v: 30


In [19]:
np.linalg.norm(list(d.values()))

37.416573867739416

In [20]:
[i if i%2==0 else i+1 for i in range(0,10) ]

[0, 2, 2, 4, 4, 6, 6, 8, 8, 10]

In [13]:
class A():
    
    def __init__(self):
        self.a = 10
        
class B(A):
    
    def __init__(self):
        super().__init__()
        self.b = 20

In [14]:
t=B()

In [16]:
t.b

20

In [17]:
w=Weighter(index, indexInverse)

In [19]:
Weighter.getStemsFromQuery("homes sales top")

['home', 'sale', 'top']

### Modèles de langues

In [99]:
class Okapi(IRModel):
    def __init__(self, weighter, k1=1.2, b=0.75):
        
        super().__init__(weighter)
        # constantes du modèle
        self.k1 = k1
        self.b = b
        
        #nombre de docs
        self.nbDoc = len(weighter.index)
        
        #longueur moyenne des documents
        self.avglen = np.mean([sum(list(index[idDoc].values())) for idDoc in range(len(docs))])
        
    def getScores(self, query):
        res={}
        #Récupère les termes de la requete
        stems = Weighter.getStemsFromQuery(query)
        #Pour chaque terme de la requete
        for stem in stems:
            #Calcul des éléments indépendant du document
            # on considère le dénominateur en 2 parties indépendantes du document (il faut développer le dénominateur):
            # 1) k1 * (1 - b)
            # 2) k1 * b / avgdl 
            idf = self.weighter.getIdf(stem)
            denom1 = self.k1 * (1-self.b) 
            denom2 = self.k1 * self.b / self.avglen #parie du dénomianteur que l'on multipliera par la longueur du doc
            
            docWeights = self.weighter.getWeightsForStem(stem)  #On récupère les poids du terme dans chaque document
            for idDoc, weight in docWeights.items():
                lenDoc = sum(self.weighter.index[idDoc].values()) #taille doc = nombre de terme avec doublon
                #lenDoc = len(self.weighter.index[idDoc]) #taille du doc = nombre de terme différent
                score = idf * weight / (weight + denom1 + denom2*lenDoc) #Formule okapi-BM25
                if(idDoc not in res):
                    res[idDoc] = score
                else:
                    res[idDoc] += score
        return res

In [112]:
w = Weighter2(index, indexInverse)
o = Okapi(w)

In [113]:
req = ("new home forecast")

In [115]:
o.getScores(req)

{0: 0.5835791788863051, 1: 0.0, 2: 0.0, 3: 0.2238678032440597}

In [82]:
index

{0: {'new': 1,
  'home': 1,
  'ha': 1,
  'been': 1,
  'sale': 1,
  'top': 1,
  'forecast': 1},
 1: {'home': 2, 'sale': 1, 'rise': 1, 'juli': 1},
 2: {'is': 1, 'increas': 1, 'home': 1, 'sale': 1, 'juli': 1},
 3: {'juli': 1, 'encount': 1, 'new': 1, 'home': 1, 'sale': 1, 'rise': 1}}